In [74]:
import json
COMMON_WORDS = {'a', 'an', 'the', 'in', 'on', 'at', 'for', 'to', 'with', 'and', 'or', 'do', 'have', 'has', 'does', 'did'}

def read_target_strings(person_id):
    file_path = f"./persona{person_id}.json"

    with open(file_path, 'r') as json_file:
        data = json.load(json_file)['data']

        BrosingHistoryTitles    = [_['title'] for _ in data['browsingHistoryList']]
        PostsContents           = [_['content'] for _ in data['facebookPostsList']]
        ScheduleAddrs           = [_['address'] for _ in data['schedule']]
        ProfileInfo             = [_ for _ in data.values() if isinstance(_, str)]
        return [BrosingHistoryTitles, PostsContents, ScheduleAddrs, ProfileInfo]

def direct_match(key_word, string_list):
    key_words_to_check = [word for word in key_word if word.lower() not in COMMON_WORDS]
    filtered_list = [
        string for string in string_list if any(
            word.lower() in string.lower().split() for word in key_words_to_check
        )
    ]
    return filtered_list

In [68]:
def sentence_vector(sentence, model):
    return sum([model.wv[word] for word in sentence if word in model.wv])

reference: https://radimrehurek.com/gensim/models/word2vec.html

In [12]:
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [69]:
def get_relevant_ws(key_words):
    similar = []
    for w in key_words:
      similar += glove_vectors.most_similar(w)
    return set([word for word, _ in similar])

In [77]:
def get_related_posts(person_id, key_word):
    tar_str = read_target_strings(person_id)
    print("------------------------------------------------")
    print("Direct Matches:")
    for lst in tar_str:
        match_strings = direct_match(key_word, lst)
        for s in match_strings:
            print(s)
    print("------------------------------------------------")
    print("Relevant Matches:")
    relevant_ws = get_relevant_ws(key_word.split())
    for lst in tar_str:
        match_strings = direct_match(relevant_ws, lst)
        for s in match_strings:
            print(s)


get_related_posts(2, 'eat a breakfast')



------------------------------------------------
Direct Matches:
Breakfast and News Reading - 1200 S Central Ave, Los Angeles, CA 90021
Breakfast and News Reading - 1200 S Central Ave, Los Angeles, CA 90021
Breakfast and News Reading - 453 S Spring St, Los Angeles, CA 90013
Work as a Financial Analyst - 601 S Figueroa St, Los Angeles, CA 90017
Lunch Break - 515 S Flower St, Los Angeles, CA 90071
Work as a Financial Analyst - 601 S Figueroa St, Los Angeles, CA 90017
Dinner - 301 S Alvarado St, Los Angeles, CA 90057
Relaxation Time - 624 S Grand Ave, Los Angeles, CA 90017
Online Gaming - 1235 S Flower St, Los Angeles, CA 90015
Watching YouTube Videos - 1150 S Olive St, Los Angeles, CA 90015
YouTube Videos - 267 S San Pedro St, Los Angeles, CA 90012
Morning Exercise - 1100 S Hope St, Los Angeles, CA 90015
Breakfast and News Reading - 1200 S Central Ave, Los Angeles, CA 90021
Commute to Work - 200 S Main St, Los Angeles, CA 90012
Commute back Home - 100 S Olive St, Los Angeles, CA 90012
Re